Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members!

In [1]:
import sqlite3
from sqlite3 import Error
 
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn
 
def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query1 = """
    SELECT * from (
    SELECT name , sum(cost) as revenue
    FROM (
    SELECT Facilities.name AS name, 
    CASE WHEN Bookings.memid =0
    THEN Facilities.guestcost * Bookings.slots
    ELSE Facilities.membercost * Bookings.slots
    END AS cost
    FROM Bookings
    INNER JOIN Facilities ON Bookings.facid = Facilities.facid
    INNER JOIN Members ON Bookings.memid = Members.memid  
    ) AS innerTable
     group by 
    name ) as outerTable
    where revenue < 1000"""
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)


def main():
    database = "sqlite_db_pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(database)
    with conn: 
        print("2. Query all tasks")
        select_all_tasks(conn)
 
 
if __name__ == '__main__':
    main()

2.6.0
2. Query all tasks
('Pool Table', 270)
('Snooker Table', 240)
('Table Tennis', 180)


Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order

In [2]:
import sqlite3
from sqlite3 import Error
 
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn
 
def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query1 = """
    SELECT (b.surname ||', '|| b.firstname) as name , (a.surname ||', '|| a.firstname) as recommendedby 
FROM `Members` a inner join `Members` b on a.memid = b.recommendedby where b.recommendedby is not null and b.recommendedby <> '' order by name;"""
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)


def main():
    database = "sqlite_db_pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(database)
    with conn: 
        print("2. Query all tasks")
        select_all_tasks(conn)
 
 
if __name__ == '__main__':
    main()

2.6.0
2. Query all tasks
('Bader, Florence', 'Stibbons, Ponder')
('Baker, Anne', 'Stibbons, Ponder')
('Baker, Timothy', 'Farrell, Jemima')
('Boothe, Tim', 'Rownam, Tim')
('Butters, Gerald', 'Smith, Darren')
('Coplin, Joan', 'Baker, Timothy')
('Crumpet, Erica', 'Smith, Tracy')
('Dare, Nancy', 'Joplette, Janice')
('Genting, Matthew', 'Butters, Gerald')
('Hunt, John', 'Purview, Millicent')
('Jones, David', 'Joplette, Janice')
('Jones, Douglas', 'Jones, David')
('Joplette, Janice', 'Smith, Darren')
('Mackenzie, Anna', 'Smith, Darren')
('Owen, Charles', 'Smith, Darren')
('Pinker, David', 'Farrell, Jemima')
('Purview, Millicent', 'Smith, Tracy')
('Rumney, Henrietta', 'Genting, Matthew')
('Sarwin, Ramnaresh', 'Bader, Florence')
('Smith, Jack', 'Smith, Darren')
('Stibbons, Ponder', 'Tracy, Burton')
('Worthington-Smyth, Henry', 'Smith, Tracy')


Q12: Find the facilities with their usage by member, but not guests

In [3]:
import sqlite3
from sqlite3 import Error
 
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn
 
def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query1 = """
SELECT sub.name, 
sub.Tennis_Court_1, 
sub.Tennis_Court_2, 
sub.Badminton_Court, 
sub.Table_Tennis, 
sub.Massage_Room_1, 
sub.Massage_Room_2, 
sub.Squash_Court, 
sub.Snooker_Table,
sub.Pool_Table
FROM (
    SELECT m.surname||' '||m.firstname as name,
    COUNT(CASE WHEN b.memid = m.memid AND f.facid=0 THEN 1 END) AS Tennis_Court_1,
    COUNT(CASE WHEN b.memid = m.memid AND f.facid=1 THEN 1 END) AS Tennis_Court_2,
    COUNT(CASE WHEN b.memid = m.memid AND f.facid=2 THEN 1 END) AS Badminton_Court,
    COUNT(CASE WHEN b.memid = m.memid AND f.facid=3 THEN 1 END) AS Table_Tennis,
    COUNT(CASE WHEN b.memid = m.memid AND f.facid=4 THEN 1 END) AS Massage_Room_1,
    COUNT(CASE WHEN b.memid = m.memid AND f.facid=5 THEN 1 END) AS Massage_Room_2,
    COUNT(CASE WHEN b.memid = m.memid AND f.facid=6 THEN 1 END) AS Squash_Court,
    COUNT(CASE WHEN b.memid = m.memid AND f.facid=7 THEN 1 END) AS Snooker_Table,
    COUNT(CASE WHEN b.memid = m.memid AND f.facid=8 THEN 1 END) AS Pool_Table
    FROM Members as m
    LEFT JOIN Bookings as b
    ON b.memid = m.memid
    LEFT JOIN Facilities as f
    ON f.facid = b.facid
    WHERE m.memid!=0
    GROUP BY m.memid) as sub """
    
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)


def main():
    database = "sqlite_db_pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(database)
    with conn: 
        print("2. Query all tasks")
        select_all_tasks(conn)
 
 
if __name__ == '__main__':
    main()

2.6.0
2. Query all tasks
('Smith Darren', 0, 19, 132, 28, 28, 0, 14, 12, 28)
('Smith Tracy', 30, 2, 32, 28, 6, 0, 6, 45, 61)
('Rownam Tim', 6, 6, 4, 69, 80, 2, 0, 0, 241)
('Joplette Janice', 19, 8, 0, 9, 12, 2, 14, 68, 27)
('Butters Gerald', 57, 3, 20, 1, 32, 1, 9, 34, 6)
('Tracy Burton', 31, 3, 2, 24, 31, 0, 35, 20, 30)
('Dare Nancy', 25, 11, 10, 5, 19, 5, 0, 23, 19)
('Boothe Tim', 4, 52, 12, 4, 36, 0, 12, 43, 25)
('Stibbons Ponder', 1, 31, 16, 3, 19, 0, 2, 20, 12)
('Owen Charles', 17, 41, 6, 24, 11, 2, 7, 22, 1)
('Jones David', 25, 30, 8, 11, 19, 4, 8, 2, 8)
('Baker Anne', 6, 35, 10, 1, 3, 2, 49, 0, 12)
('Farrell Jemima', 1, 1, 7, 12, 29, 0, 8, 21, 1)
('Smith Jack', 22, 1, 12, 5, 27, 1, 9, 5, 7)
('Bader Florence', 1, 8, 9, 42, 0, 2, 2, 33, 23)
('Baker Timothy', 14, 7, 7, 24, 24, 0, 5, 0, 85)
('Pinker David', 16, 0, 7, 17, 3, 0, 3, 16, 9)
('Genting Matthew', 1, 0, 0, 26, 25, 1, 0, 1, 18)
('Mackenzie Anna', 0, 0, 30, 16, 1, 0, 2, 7, 70)
('Coplin Joan', 7, 0, 0, 21, 1, 2, 1, 10, 11)
('S

Q13: Find the facilities usage by month, but not guests

In [4]:
import sqlite3
from sqlite3 import Error
 
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn
 
def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query1 = """
SELECT sub.name, sub.July_usage, sub.August_usage, sub.Sept_usage
FROM
(SELECT 
COUNT(CASE WHEN b.memid!=0 AND b.starttime LIKE '%07%' THEN 1 END) as July_usage,
COUNT(CASE WHEN b.memid!=0 AND b.starttime LIKE '%08%' THEN 1 END) as August_usage,
COUNT(CASE WHEN b.memid!=0 AND b.starttime LIKE '%09%' THEN 1 END) as Sept_usage,
f.name
FROM Bookings as b
LEFT JOIN Facilities as f
ON f.facid = b.facid
GROUP BY f.name) as sub """
    
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)


def main():
    database = "sqlite_db_pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(database)
    with conn: 
        print("2. Query all tasks")
        select_all_tasks(conn)
 
 
if __name__ == '__main__':
    main()

2.6.0
2. Query all tasks
('Badminton Court', 60, 166, 186)
('Massage Room 1', 87, 189, 209)
('Massage Room 2', 6, 11, 15)
('Pool Table', 125, 323, 438)
('Snooker Table', 77, 190, 220)
('Squash Court', 25, 94, 99)
('Table Tennis', 58, 178, 211)
('Tennis Court 1', 70, 143, 152)
('Tennis Court 2', 49, 142, 142)
